# ETL Class Example Notebook

This notebook demonstrates the three main steps of the ETL pipeline: **Extract**, **Transform**, and **Load**. It mirrors the structure of `etl_notebook.ipynb` but provides a concise class‑based example for quick reference.


## Extract

Set up the project root on `sys.path` so that package imports work from any working directory. Import the extraction utilities required for this example.


In [6]:
import os
import sys
from datetime import datetime, timezone
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.dialects.postgresql import insert
from sqlalchemy.orm import Session

In [11]:

from ca_biositing.datamodels.config import settings
from ca_biositing.pipeline.etl.extract import proximate, samplemetadata, ultimate, cmpana, static_resource_info 

# Extract data
samplemetadata_raw = samplemetadata.extract()   
static_resource_info_raw = static_resource_info.extract()
#prox_raw = proximate.extract()
#ult_raw = ultimate.extract()
#cmpana_raw = cmpana.extract()

#feedstock_collector_info_raw = feedstock_collection_info.extract()

#sampleids_raw = sample_ids.extract()

#analysis_data = [prox_raw, ult_raw, cmpana_raw]

static_resource_info_raw.dtypes

13:00:49.964 | INFO    | Task run 'extract' - Extracting raw data from 'samplemetadata' in 'Sampling_data_redacted'...

DEBUG: gsheet_to_df called for Sampling_data_redacted / samplemetadata
DEBUG: Authenticating with credentials.json
DEBUG: Opening spreadsheet Sampling_data_redacted
DEBUG: Opening worksheet samplemetadata
DEBUG: Fetching all values from samplemetadata
DEBUG: Successfully fetched 106 rows


13:00:51.195 | INFO    | Task run 'extract' - Successfully extracted raw data.

13:00:51.198 | INFO    | Task run 'extract' - Finished in state Completed()

13:00:51.222 | INFO    | Task run 'extract' - Extracting raw data from 'static_resource_info' in 'Static_resource_information'...

DEBUG: gsheet_to_df called for Static_resource_information / static_resource_info
DEBUG: Authenticating with credentials.json
DEBUG: Opening spreadsheet Static_resource_information
DEBUG: Opening worksheet static_resource_info
DEBUG: Fetching all values from static_resource_info
DEBUG: Successfully fetched 95 rows


13:00:52.426 | INFO    | Task run 'extract' - Successfully extracted raw data.

13:00:52.430 | INFO    | Task run 'extract' - Finished in state Completed()

Resource                      object
Resource Code                 object
LandIQ Crop Name              object
Residue Type                  object
Collected?                    object
From Month                    object
To Month                      object
Residue Yield (Wet Ton/Ac)    object
Moisture Content              object
Residue Yield (Dry Ton/Ac)    object
dtype: object

## Transform


In [ ]:
from ca_biositing.pipeline.utils.cleaning_functions import cleaning as cleaning_mod

cleaned_data = []
for df in analysis_data:
    df['dataset'] = 'biocirv'
    cleaned_df = cleaning_mod.standard_clean(df)
    cleaned_data.append(cleaned_df)

print(f"Cleaned {len(cleaned_data)} dataframes.")

In [ ]:
from ca_biositing.pipeline.utils.cleaning_functions import coercion as coercion_mod

coerced_data = []
for df in cleaned_data:
    # Example: coerce columns into the designated data types (int, float, datetime, geom, etc)
    coerced_df = coercion_mod.coerce_columns(df,
                                             int_cols=['repl_no'], 
                                             float_cols=['value'],
                                             datetime_cols=['created_at', 'updated_at'])
    coerced_data.append(coerced_df)

print(f"Coerced {len(coerced_data)} dataframes.")

In [ ]:
from ca_biositing.pipeline.utils.name_id_swap import normalize_dataframes
from ca_biositing.datamodels.schemas.generated.ca_biositing import *

normalize_columns = {
    'resource': (Resource, 'name'),
    'prepared_sample': (PreparedSample, 'name'),
    'preparation_method': (Method, 'name'),
    'parameter': (Parameter, 'name'),
    'unit': (Unit, 'name'),
    'sample_unit': (Unit, 'name'),
    'analyst_email': (Contact, 'email'),
    'primary_ag_product': (PrimaryAgProduct, 'name'),
    'provider_code': (Provider, 'codename'),
    'dataset': (Dataset, 'name')
}

normalized_data = []
for df in coerced_data:
    normalized_df = normalize_dataframes(df, normalize_columns)
    normalized_data.append(normalized_df)

print(f"Normalized {len(normalized_data)} dataframes.")

In [ ]:
## Prepare Record Information DataFrames

observation_data = []
for df in normalized_data:
    obs_df = df[[
        'dataset_id',
        'analysis_type', 
        'record_id',
        'parameter_id',
        'value',
        'unit_id', 
        'note'
    ]].copy().rename(columns={'analysis_type': 'record_type'})
    obs_df = obs_df.dropna(subset=['record_id', 'parameter_id', 'value'])
    observation_data.append(obs_df)

print(f"Prepared {len(observation_data)} observation dataframes.")

In [ ]:
## Prepare Record Information DataFrames

record_data = []
for df in normalized_data:
    # 1. Define explicit mappings for non-normalized columns
    rename_map = {
        'record_id': 'record_id',
        'repl_no': 'technical_replication_no',
        'qc_result': 'qc_pass',
        'note': 'note'
    }
    
    # 2. Dynamically add normalized columns from the normalize_columns dictionary
    for col in normalize_columns.keys():
        norm_col = f"{col}_id"
        if norm_col in df.columns:
            # Special case: rename to match target record table schema
            target_name = 'analyst_id' if col == 'analyst_email' else \
                          'method_id' if col == 'preparation_method' else norm_col
            rename_map[norm_col] = target_name

    # 3. Only select columns that actually exist in this specific dataframe
    available_cols = [c for c in rename_map.keys() if c in df.columns]
    final_rename = {k: v for k, v in rename_map.items() if k in available_cols}

    record_df = df[available_cols].copy().rename(columns=final_rename)

    # 4. Drop rows where critical identifiers are missing (NaN)
    if 'record_id' in record_df.columns:
        record_df = record_df.dropna(subset=['record_id'])
    
    record_data.append(record_df)

print(f"Prepared {len(record_data)} record dataframes.")

## Load


In [ ]:
db_url = settings.database_url
if "@db:" in db_url:
    db_url = db_url.replace("@db:", "@localhost:")
elif "db:5432" in db_url:
    db_url = db_url.replace("db:5432", "localhost:5432")

engine = create_engine(db_url)

def upsert_observations(df, session):
    if df.empty:
        return
    now = datetime.now(timezone.utc)
    records = df.replace({np.nan: None}).to_dict(orient='records')
    for record in records:
        record['updated_at'] = now
        if record.get('created_at') is None:
            record['created_at'] = now
        stmt = insert(Observation).values(record)
        update_dict = {
            c.name: stmt.excluded[c.name]
            for c in Observation.__table__.columns
            if c.name not in ['id', 'created_at', 'record_id']
        }
        upsert_stmt = stmt.on_conflict_do_update(
            index_elements=['record_id'],
            set_=update_dict
        )
        session.execute(upsert_stmt)

with Session(engine) as session:
    for obs_df in observation_data:
        upsert_observations(obs_df, session)
    session.commit()
print('Upsert of all observations completed.')

In [ ]:
from ca_biositing.datamodels.schemas.generated.ca_biositing import ProximateRecord

def upsert_proximate_records(df, session):
    if df.empty:
        return
    now = datetime.now(timezone.utc)
    # Filter record dictionary to only include columns that exist in the table
    table_columns = {c.name for c in ProximateRecord.__table__.columns}
    records = df.replace({np.nan: None}).to_dict(orient='records')
    for record in records:
        clean_record = {k: v for k, v in record.items() if k in table_columns}
        clean_record['updated_at'] = now
        if clean_record.get('created_at') is None:
            clean_record['created_at'] = now
        stmt = insert(ProximateRecord).values(clean_record)
        update_dict = {
            c.name: stmt.excluded[c.name]
            for c in ProximateRecord.__table__.columns
            if c.name not in ['id', 'created_at', 'record_id']
        }
        upsert_stmt = stmt.on_conflict_do_update(
            index_elements=['record_id'],
            set_=update_dict
        )
        session.execute(upsert_stmt)

with Session(engine) as session:
    # Assuming the first dataframe in record_data is Proximate
    upsert_proximate_records(record_data[0], session)
    session.commit()
print('Upsert of Proximate records completed.')

In [ ]:
from ca_biositing.datamodels.schemas.generated.ca_biositing import UltimateRecord

def upsert_ultimate_records(df, session):
    if df.empty:
        return
    now = datetime.now(timezone.utc)
    table_columns = {c.name for c in UltimateRecord.__table__.columns}
    records = df.replace({np.nan: None}).to_dict(orient='records')
    for record in records:
        clean_record = {k: v for k, v in record.items() if k in table_columns}
        clean_record['updated_at'] = now
        if clean_record.get('created_at') is None:
            clean_record['created_at'] = now
        stmt = insert(UltimateRecord).values(clean_record)
        update_dict = {
            c.name: stmt.excluded[c.name]
            for c in UltimateRecord.__table__.columns
            if c.name not in ['id', 'created_at', 'record_id']
        }
        upsert_stmt = stmt.on_conflict_do_update(
            index_elements=['record_id'],
            set_=update_dict
        )
        session.execute(upsert_stmt)

with Session(engine) as session:
    # Assuming the second dataframe in record_data is Ultimate
    upsert_ultimate_records(record_data[1], session)
    session.commit()
print('Upsert of Ultimate records completed.')

In [ ]:
from ca_biositing.datamodels.schemas.generated.ca_biositing import CompositionalRecord

def upsert_compositional_records(df, session):
    if df.empty:
        return
    now = datetime.now(timezone.utc)
    table_columns = {c.name for c in CompositionalRecord.__table__.columns}
    records = df.replace({np.nan: None}).to_dict(orient='records')
    for record in records:
        clean_record = {k: v for k, v in record.items() if k in table_columns}
        clean_record['updated_at'] = now
        if clean_record.get('created_at') is None:
            clean_record['created_at'] = now
        stmt = insert(CompositionalRecord).values(clean_record)
        update_dict = {
            c.name: stmt.excluded[c.name]
            for c in CompositionalRecord.__table__.columns
            if c.name not in ['id', 'created_at', 'record_id']
        }
        upsert_stmt = stmt.on_conflict_do_update(
            index_elements=['record_id'],
            set_=update_dict
        )
        session.execute(upsert_stmt)

with Session(engine) as session:
    # Assuming the third dataframe in record_data is Compositional
    upsert_compositional_records(record_data[2], session)
    session.commit()
print('Upsert of Compositional records completed.')

In [ ]:
feedstock_collector_info_raw

In [ ]:
from ca_biositing.pipeline.etl.extract.biodiesel_plants import extract as biodiesel_extract
biodiesel_plants_raw = biodiesel_extract()

In [ ]:
biodiesel_plants_raw.head()

In [ ]:
from typing import List
from ca_biositing.pipeline.etl.extract.basic_sample_info import extract as basic_sample_info_extract
from ca_biositing.pipeline.etl.transform.resource import transform as resource_transform

basic_sample_info = basic_sample_info_extract()

EXTRACT_SOURCES: List[str] = ["basic_sample_info"]

cleaned_data = resource_transform.fn({"basic_sample_info": basic_sample_info})

cleaned_data.head()